In [1]:
import xarray as xr
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from pyproj import Transformer, CRS, Proj

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

SELECTED_LOCATION = "Cordova"
YEAR = 1991 # picked to avoid any leap year stuff possibly confounding the analysis
SOURCE_DIR = Path(f"/beegfs/CMIP6/wrf_era5/04km/{YEAR}")

### change this to where your outputs are
PROCESSED_FILE = Path(f"/beegfs/CMIP6/cparr4/daily_downscaled_era5_for_rasdaman/t2_mean/t2_mean_{YEAR}_daily_era5_4km_3338.nc")

ak_locations = {
    "Anchorage": (61.2181, -149.9003),
    "Fairbanks": (64.8378, -147.7164),
    "Utqiaġvik": (71.2906, -156.7886),
    "Bethel": (60.7922, -161.7558),
    "Cordova": (60.5438, -145.7573),
    "Nome": (64.5011, -165.4064),
    "Seward": (60.1044, -149.4458),
    "WRF": (64.0, -152.0)
}

source_files = sorted(SOURCE_DIR.glob(f"era5_wrf_dscale_4km_{YEAR}-*.nc"))
ds_processed = xr.open_dataset(PROCESSED_FILE)

In [2]:
def find_nearest_grid_indices(ds, locations):
    """Find the nearest grid indices for a set of lat/lon locations."""
    ak_grid_indices = {}
    lats = ds['XLAT'].values
    lons = ds['XLONG'].values
    for name, (lat, lon) in locations.items():
        # Compute squared distance for all grid points
        dist2 = (lats - lat)**2 + (lons - lon)**2
        idx = np.unravel_index(np.argmin(dist2), lats.shape)
        ak_grid_indices[name] = {'south_north': idx[0], 'west_east': idx[1]}
    return ak_grid_indices

def project_locations(locations_lat_lon):
    """Project lat/lon coordinates to EPSG:3338."""
    to_ak_albers = Transformer.from_crs("EPSG:4326", "EPSG:3338", always_xy=True)
    projected_locs = {name: to_ak_albers.transform(lon, lat) for name, (lat, lon) in locations_lat_lon.items()}
    return projected_locs

ak_locations_3338 = project_locations(ak_locations)
ak_locations_3338

{'Anchorage': (219349.5792197109, 1255301.5397254487),
 'Fairbanks': (297698.8056800067, 1667062.2461945596),
 'Utqiaġvik': (-102347.93849687307, 2368027.864846956),
 'Bethel': (-419835.81379570364, 1225436.359509806),
 'Cordova': (449500.6129700059, 1201041.5176492375),
 'Nome': (-544971.6990630961, 1662325.0658296624),
 'Seward': (252166.09829785314, 1132616.5920323776),
 'WRF': (97696.46311357869, 1560949.558784527)}

In [3]:
print("Finding nearest grid cell in the first source file...")
with xr.open_dataset(source_files[0]) as sample_ds:
    grid_indices = find_nearest_grid_indices(sample_ds, ak_locations)

source_loc_indices = grid_indices[SELECTED_LOCATION]
print(source_loc_indices)
print("get the **exact** lat-lon for that grid cell")
exact_lat = float(sample_ds['XLAT']
                  .isel(south_north=source_loc_indices['south_north'],
                       west_east=source_loc_indices['west_east']))
exact_lon = float(sample_ds['XLONG']
                  .isel(south_north=source_loc_indices['south_north'],
                       west_east=source_loc_indices['west_east']))
print(exact_lat, exact_lon)
print("notice that this is different than our initial input lat-lon!")
print(ak_locations[SELECTED_LOCATION])
print("project both to 3338, and compute the distance between the two points")
to_ak_albers = Transformer.from_crs("EPSG:4326", "EPSG:3338", always_xy=True)
x_exact_3338, y_exact_3338 = to_ak_albers.transform(exact_lon, exact_lat)
def distance_m_3338(p1, p2):
    """Euclidean distance between two EPSG:3338 points, in metres.

    Parameters
    ----------
    p1, p2
        Two-element sequences of *(x, y)* in metres (Alaska Albers).

    Returns
    -------
    float
        Distance in metres.
    """
    from math import hypot
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return hypot(dx, dy) 

dist_m  = distance_m_3338(ak_locations_3338[SELECTED_LOCATION], (x_exact_3338, y_exact_3338))
dist_km = dist_m / 1000
print(f"{dist_km:.1f} km")

Finding nearest grid cell in the first source file...
{'south_north': 135, 'west_east': 299}
get the **exact** lat-lon for that grid cell
60.53725814819336 -145.77389526367188
notice that this is different than our initial input lat-lon!
(60.5438, -145.7573)
project both to 3338, and compute the distance between the two points
1.2 km


In [4]:
print(f"Processing comparison for location: {SELECTED_LOCATION}")
# brute force search the surrounding grid cells up to NxN offset to find best match
# it should be in there somewhere!

daily_means_dict = { (di,dj): [] for di in [-3,-2,-1,0,1,2,3] for dj in [-3,-2,-1,0,1,2,3] }
print(f"Processing {len(source_files)} source files in a loop for multiple offsets...")
for f in source_files:
    # could use mf data open here, but this is fast enough
    with xr.open_dataset(f) as ds:
        for di,dj in daily_means_dict.keys():
            wn = source_loc_indices['west_east'] + di
            sn = source_loc_indices['south_north'] + dj
            
            wn = max(0, min(wn, ds.dims['west_east']-1))
            sn = max(0, min(sn, ds.dims['south_north']-1))
            
            source_raw = ds['T2'].isel(west_east=wn, south_north=sn)
            daily_mean = source_raw.resample(Time="1D").mean() - 273.15
            daily_means_dict[(di,dj)].append(daily_mean)
            
print("Combining daily means for each offset...")
offset_series = {}
for key, lst in daily_means_dict.items():
    series = xr.concat(lst, dim="Time").rename({'Time':'time'}).rename("t2_mean_source")
    offset_series[key] = series

print("Extracting data from processed file...")
processed_loc_coords = ak_locations_3338[SELECTED_LOCATION]
processed_daily_mean = ds_processed["t2_mean"].sel(
    ###
    # swap the commented out parts here to see the offset location with the min delta shift
    x=x_exact_3338,
    y=y_exact_3338,
    #x=processed_loc_coords[0],
    #y=processed_loc_coords[1],
    ###
    method="nearest"
)

print("\nOffset summary (mean absolute delta °C):")
delta_dict = {}
for key, src_series in offset_series.items():
    aligned_src, aligned_proc = xr.align(src_series, processed_daily_mean, join="inner")
    d = aligned_proc - aligned_src
    delta_dict[key] = float(np.abs(d).mean())
    print(f"  offset {key}: {delta_dict[key]:.2f}")

# pick best offset (minimum mean abs delta)
best_offset = min(delta_dict, key=delta_dict.get)
print(f"\nBest offset: {best_offset} with mean abs delta {delta_dict[best_offset]:.2f} °C")

# Use best offset for detailed plotting
aligned_source = offset_series[best_offset]
aligned_source, aligned_processed = xr.align(aligned_source, processed_daily_mean, join="inner")
delta = aligned_processed - aligned_source
delta = delta.rename("t2_mean_delta")

print("Generating plot for best offset...")

aligned_source = aligned_source.reset_coords(drop=True)
aligned_processed = aligned_processed.reset_coords(drop=True)
delta = delta.reset_coords(drop=True)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 10), sharex=True)

aligned_source.plot(ax=axes[0], label='Source (Daily Mean)')
axes[0].set_title(f'Source Daily Mean Temperature at {SELECTED_LOCATION}')
axes[0].set_ylabel('Temperature (°C)')
axes[0].grid(True)

aligned_processed.plot(ax=axes[1], color='orange', label='Processed (Daily Mean)')
axes[1].set_title(f'Processed Daily Mean Temperature at {SELECTED_LOCATION}')
axes[1].set_ylabel('Temperature (°C)')
axes[1].grid(True)

delta.plot(ax=axes[2], color='green', label='Delta (Processed - Source)')
axes[2].axhline(0, color='red', linestyle='--')
delta_min = float(delta.min())
delta_max = float(delta.max())
axes[2].set_ylim(delta_min, delta_max)

axes[2].set_title('Difference (Processed - Source)')
axes[2].set_ylabel('Temperature Delta (°C)')
axes[2].grid(True)

fig.suptitle(f'Temperature Comparison for {SELECTED_LOCATION} - {YEAR}', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

output_fig_path = f"qc_comparison_{SELECTED_LOCATION}_{YEAR}.png"
plt.savefig(output_fig_path)
print(f"Plot saved to {output_fig_path}")
plt.close()

print(f"Mean Difference: {delta.mean().item():.2f} °C")
print(f"Max Difference: {delta.max().item():.2f} °C")
print(f"Min Difference: {delta.min().item():.2f} °C")

Processing comparison for location: Cordova
Processing 365 source files in a loop for multiple offsets...
Combining daily means for each offset...
Extracting data from processed file...

Offset summary (mean absolute delta °C):
  offset (-3, -3): 1.35
  offset (-3, -2): 1.11
  offset (-3, -1): 0.67
  offset (-3, 0): 1.47
  offset (-3, 1): 1.62
  offset (-3, 2): 1.13
  offset (-3, 3): 1.56
  offset (-2, -3): 1.23
  offset (-2, -2): 1.24
  offset (-2, -1): 0.86
  offset (-2, 0): 1.52
  offset (-2, 1): 1.63
  offset (-2, 2): 0.96
  offset (-2, 3): 1.56
  offset (-1, -3): 1.02
  offset (-1, -2): 1.06
  offset (-1, -1): 1.33
  offset (-1, 0): 1.65
  offset (-1, 1): 1.69
  offset (-1, 2): 1.65
  offset (-1, 3): 1.48
  offset (0, -3): 1.11
  offset (0, -2): 0.00
  offset (0, -1): 1.81
  offset (0, 0): 1.49
  offset (0, 1): 1.66
  offset (0, 2): 1.94
  offset (0, 3): 1.44
  offset (1, -3): 1.29
  offset (1, -2): 0.58
  offset (1, -1): 1.89
  offset (1, 0): 1.72
  offset (1, 1): 1.41
  offset (

In [5]:
# lat / lon that came from WRF (they were computed on a sphere)
lat_sph, lon_sph = exact_lat, exact_lon

# stock EPSG:3338 ellipsoid, not a sphere
ak_albers_ellip = CRS.from_epsg(3338)
to_ellip = Transformer.from_crs(4326, ak_albers_ellip, always_xy=True)
x_ellip, y_ellip = to_ellip.transform(lon_sph, lat_sph)

# *Spherical* Alaska Albers with the same radius WRF uses, basically an imaginary CRS
ak_albers_sphere = CRS.from_proj4(
    "+proj=aea +lat_1=55 +lat_2=65 +lat_0=50 +lon_0=-154 "
    "+x_0=0 +y_0=0 +a=6370000 +b=6370000 +units=m +no_defs"
)
to_sphere = Transformer.from_crs(4326, ak_albers_sphere, always_xy=True)
x_sph, y_sph = to_sphere.transform(lon_sph, lat_sph)

# Northward datum shift
dy = y_ellip - y_sph
print(f"Northward datum shift ≈ {dy/4000:.2f} grid rows "
      f"({dy:,.1f} m for 4 km cells)")

Northward datum shift ≈ 0.42 grid rows (1,689.4 m for 4 km cells)


In [6]:
# recreating logic from the pipeline here

wrf_proj = "+proj=stere +units=m +a=6370000.0 +b=6370000.0 +lat_0=90.0 +lon_0=-152 +lat_ts=64 +nadgrids=@null"
wgs_proj = Proj(proj="latlong", datum="WGS84")
wgs_to_wrf_transformer = Transformer.from_proj(wgs_proj, wrf_proj)

with xr.open_dataset(source_files[0]) as tmp_ds:
    nx = tmp_ds.XLONG.shape[1]
    ny = tmp_ds.XLONG.shape[0]
    
with xr.open_dataset("/beegfs/CMIP6/wrf_era5/geo_em.d02.nc") as geo_ds:
    # this is where we plug in the center longitude of the domain to get the center x, y in projected space
    e, n = wgs_to_wrf_transformer.transform(
        geo_ds.attrs["CEN_LON"], geo_ds.attrs["TRUELAT1"]
    )

dx = dy = 4000
x0 = -(nx - 1) / 2.0 * dx + e
y0 = -(ny - 1) / 2.0 * dy + n

x = np.arange(nx) * dx + x0
y = np.arange(ny) * dy + y0
wrf_crs = CRS.from_proj4(wrf_proj)

ds_proj = (
    ds.rename({"south_north": "y", "west_east": "x"})
    .assign_coords({"y": ("y", y), "x": ("x", x)})
    .drop_vars(["XLONG", "XLAT"])
    .rio.set_spatial_dims("x", "y")
    .rio.write_crs(wrf_crs)
)
# pipeline would then reproject via rioxarray, but we skip that here
# end pipeline mimic

# centre y-coords you attached to ds_proj
y_centres = ds_proj.y.values

# upper-left pixel edge that GDAL infers
from rasterio.transform import from_origin
aff = ds_proj.rio.transform() # affine (edge-referenced)
edge_y0 = aff.f # y of the first pixel edge

print(f"centre y[0]  = {y_centres[0]:.1f} m")
print(f"edge   y0    = {edge_y0:.1f} m")
print(f"Δy = {abs(y_centres[0] - edge_y0):.1f} m "
      f"({abs(y_centres[0] - edge_y0)/4000:.2f} rows)")


centre y[0]  = -3690393.7 m
edge   y0    = -3692393.7 m
Δy = 2000.0 m (0.50 rows)


In [8]:
# explore "target grid anchoring"
import numpy as np

# ------------------------------------------------------------
# INPUTS you already have at this stage
# ------------------------------------------------------------
# x_site, y_site   → the EPSG:3338 coordinates of your location
# ds_3338          → the re-projected Dataset (after .rio.reproject)

# ------------------------------------------------------------
# 1. Grab the target affine transform (edge-referenced)
# ------------------------------------------------------------
aff = ds_processed.rio.transform()     # x = c + a·col + b·row ;  y = f + d·col + e·row
pix_w  =  aff.a                   # > 0 (metres)   – pixel width
pix_h  = -aff.e                   # > 0            – pixel height; aff.e is negative
edge_x0, edge_y0 = aff.c, aff.f   # upper-left pixel **edge** in metres

# ------------------------------------------------------------
# 2. Compute the *floating* row / col where the point falls
#    (row 0, col 0 are the first pixel **edges**)
# ------------------------------------------------------------
row_f = (edge_y0 - y_site) / pix_h        # northing → row
col_f = (x_site   - edge_x0) / pix_w      # easting  → col

# ------------------------------------------------------------
# 3. Distance from the nearest pixel *centre*
#    pixel centres sit at … , 0.5, 1.5, 2.5, …
# ------------------------------------------------------------
row_frac = row_f % 1.0                    # fractional part ∈ [0,1)
col_frac = col_f % 1.0

# anchor offsets in units of “rows” / “cols” (+ north / east)
row_anchor = 0.5 - row_frac               # ↑  +0.50  means ½-row northward
col_anchor = col_frac - 0.5               # →  +0.50  means ½-col eastward

print(f"Target-grid anchor (north) : {row_anchor:+.2f} rows "
      f"({row_anchor*pix_h: .1f} m)")
print(f"Target-grid anchor (east)  : {col_anchor:+.2f} cols "
      f"({col_anchor*pix_w: .1f} m)")


NameError: name 'y_site' is not defined

In [ ]:
# ───────────────────────────────────────────────────────────────────────
# Diagnostic:  target-grid anchoring (how GDAL's new grid origin nudges you)
# -----------------------------------------------------------------------
#  1. Get the affine transform of the *processed* cube (edge-referenced).
#  2. Convert the site's x/y (metres) into floating-point row/col indexes.
#  3. Compare to the nearest pixel *centre* (… , 0.5, 1.5, 2.5, …).
#  4. Report the fractional offset in "rows" (north +, south −).
# ───────────────────────────────────────────────────────────────────────
import numpy as np

# 1) edge-based affine:  x = c + a·col   ;   y = f + e·row   (e < 0 for north-up)
aff = ds_processed.rio.transform()

pix_w = aff.a                    # pixel width  (metres, positive)
pix_h = abs(aff.e)               # pixel height (metres, positive)
edge_x0, edge_y0 = aff.c, aff.f  # x/y of **upper-left pixel edge**

# 2) floating row / col where the site falls (row 0, col 0 = upper-left edge)
row_f = (edge_y0 - y_exact_3338) / pix_h   # northing → row (north-up ⇒ minus sign)
col_f = (x_exact_3338 - edge_x0) / pix_w   # easting  → col

# 3) distance from the nearest pixel *centre* (centre grid is 0.5, 1.5, …)
row_frac = row_f % 1.0
col_frac = col_f % 1.0

row_anchor = 0.5 - row_frac      # +0.50 ⇒ centre lies ½-row **north** of edge grid
col_anchor = col_frac - 0.5      # +0.50 ⇒ centre lies ½-col **east**  of edge grid

# 4) print the meridional (north/south) component in rows & metres
print(
    f"Target-grid anchor: {row_anchor:+.2f} rows "
    f"({row_anchor * pix_h: .1f} m north (+)/south (−) of pixel centre)"
)


In [ ]:
# ──────────────────────────────────────────────────────────────
# Demonstrate nearest-neighbour rounding in the processed raster
# ──────────────────────────────────────────────────────────────
import numpy as np

aff = ds_processed.rio.transform()          # edge-based affine
pix_w = aff.a                               # pixel width  (metres, +)
pix_h = abs(aff.e)                          # pixel height (metres, +)
edge_x0, edge_y0 = aff.c, aff.f             # upper-left pixel edge

# 1) floating-point row / col where the point lands (row=0 at N edge)
row_f = (edge_y0 - y_exact_3338) / pix_h
col_f = (x_exact_3338 - edge_x0) / pix_w

# 2) nearest-neighbour row / col by plain rounding (0.5 → up)
row_nn = int(np.floor(row_f + 0.5))
col_nn = int(np.floor(col_f + 0.5))

print(f"Floating indices  : row = {row_f:.3f} , col = {col_f:.3f}")
print(f"Nearest neighbour : row = {row_nn} , col = {col_nn}")

# 3) let xarray do the selection with 'method=\"nearest\"'
val_nn = ds_processed["t2_mean"].sel(x=x_exact_3338,
                                     y=y_exact_3338,
                                     method="nearest")

# map the returned x/y back onto integer row / col
row_sel = np.abs(ds_processed.y - val_nn.y).argmin().item()
col_sel = np.abs(ds_processed.x - val_nn.x).argmin().item()

print(f"xarray selected   : row = {row_sel} , col = {col_sel}")

# # 4) confirm they match
# print(f"Does plain rounding reproduce xarray’s choice?  "
#       f\"{'YES' if (row_nn, col_nn)==(row_sel, col_sel) else 'NO'}\")

# # 5) how big was the rounding jump (in rows)?
# print(f"Nearest-neighbour row jump = {(row_nn - row_f):+.2f} rows "
#       f\"({(row_nn - row_f)*pix_h: .1f} m)\")
